In [1]:
# Imports

import cv2
from PIL import Image

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import math as maths

import os
import random

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.nn import functional
#from torchsummary import summary
#import torchvision.transforms as transforms

from tqdm import tqdm
from IPython.display import clear_output

import wandb
import pprint


from functions import import_imagedata, get_data, label_oh_tf,  Unwrap, ImageProcessor, IDSWDataSetLoader
from architectures import vgg16net, smallnet1, smallnet2, smallnet3
from loop_fns import loop, test_loop
from fns4wandb import build_optimizer, set_optimizer, hp_sweep, train_model, train_log, log_test_score, set_lossfn, pipeline

from architectures import build_net, smallnet3


In [2]:
"""config = {
    'method': 'random',
    'metric':{
        'goal': 'minimize',
        'name': 'val_loss'},
    'parameters': {
        #'dropout':{
        #    'values': [0.5, 0.4, 0.3]
        #},
        'weight_decay':{
            'values': [1e-5,2e-5, 3e-5,4e-5]
        },
        'epochs':{
            'value': 3
        },
        'lin_layer_size': {
            'values': [100] #, 150, 50
        },
        'first_lin_lay':{
            'values':[248832]
        },
        'optimizer': {
            'values': ['adam']
        },
            'learning_rate': {
                # a flat distribution between 0 and 0.1
                'distribution': 'log_uniform_values',
                'min': 1e-5,
                'max': 1e-2
            },
        'loss_fn': {
            'values': ['CrossEntropy'] #'MSE', 
        },
        'data_set':{
            'values':['Augmented']
        },
            'scheduler': {
            'values': [0.1, 0.01, 0.001]
        },
        'ks': {
            'values': [(3,5)]
        },
        'dropout':{
            'values': [0.5, 0.4, 0.3, 0.2, 0.7]
        },
        'channels':{
            'values': [3]
        },
        'num_classes': {
            'values': [11]
        },
        'model_name' : {'values': ['vgg16net']},
        'channels' : {'values': [3]},
        'image_path': {
            'values': [r'/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/']
        },
        }
    }

col_dict = {
    'colour': 'colour',
    'size': [452,144],
    'padding': 5
}

title = f'BATCHTEST_IDSW_7c3l_150_122023'
save_dict = {'Run' : title,
            'Current_Epoch': 0,
            'save_location' : r'pickles/'}
#pickles


sweep_id = wandb.sweep(config, project=f"modelSize_7conv3lin_IDSW_{col_dict['colour']}_{col_dict['size']}_{col_dict['padding']}")"""

'config = {\n    \'method\': \'random\',\n    \'metric\':{\n        \'goal\': \'minimize\',\n        \'name\': \'val_loss\'},\n    \'parameters\': {\n        #\'dropout\':{\n        #    \'values\': [0.5, 0.4, 0.3]\n        #},\n        \'weight_decay\':{\n            \'values\': [1e-5,2e-5, 3e-5,4e-5]\n        },\n        \'epochs\':{\n            \'value\': 3\n        },\n        \'lin_layer_size\': {\n            \'values\': [100] #, 150, 50\n        },\n        \'first_lin_lay\':{\n            \'values\':[248832]\n        },\n        \'optimizer\': {\n            \'values\': [\'adam\']\n        },\n            \'learning_rate\': {\n                # a flat distribution between 0 and 0.1\n                \'distribution\': \'log_uniform_values\',\n                \'min\': 1e-5,\n                \'max\': 1e-2\n            },\n        \'loss_fn\': {\n            \'values\': [\'CrossEntropy\'] #\'MSE\', \n        },\n        \'data_set\':{\n            \'values\':[\'Augmented\']\n      

In [8]:

config = dict(
    epochs= 80, 
    learning_rate =3.0779e-5,
    dataset= 'IDSW',
    architecture ='CNN',
    optimizer= 'adam',
    weight_decay= 4e-5,
    dropout = 0.4,
    first_lin_lay = 248832 #3981312, #248832,
    lin_layer_size= 100,
    ks = (3,5),
    in_chan = 3,
    image_path = r'/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/',
    model_name = 'vgg16net',
    channels = 3,
    num_classes = 11,
    loss_fn = 'CrossEntropy',
    scheduler = 0.01
)



col_dict = {
    'colour': 'colour',
    'size': [452,144],
    'padding': 5
}

title = f'BATCHTEST_IDSW_7c3l_150_122023'
save_dict = {'Run' : title,
            'Current_Epoch': 0,
            'save_location' : r'pickles/'}

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2726140858.py, line 9)

In [4]:
import pprint 
pp = pprint.PrettyPrinter()
pp.pprint(config)
print(type(config))
pp.pprint(col_dict)
print(type(col_dict))

c = col_dict['colour']
print(c)

{'architecture': 'CNN',
 'channels': 3,
 'dataset': 'IDSW',
 'dropout': 0.4,
 'epochs': 80,
 'first_lin_lay': 3981312,
 'image_path': '/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/',
 'in_chan': 3,
 'ks': (3, 5),
 'learning_rate': 3.0779e-05,
 'lin_layer_size': 100,
 'loss_fn': 'CrossEntropy',
 'model_name': 'vgg16net',
 'num_classes': 11,
 'optimizer': 'adam',
 'scheduler': 0.01,
 'weight_decay': 4e-05}
<class 'dict'>
{'colour': 'colour', 'padding': 5, 'size': [452, 144]}
<class 'dict'>
colour


In [5]:
import os
print(os.getcwd())

/its/home/nn268/antvis/antvis/optics


In [6]:
#pprint.pprint(config)

In [9]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model= pipeline(config, col_dict, title, device, image_file_path= r'/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/')

train model, col dict is a <class 'dict'>


  0%|                               | 0/80 [00:00<?, ?it/s]/its/home/nn268/.local/lib/python3.10/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
  0%|                               | 0/80 [00:00<?, ?it/s]

pre loop, col_dict is a:  <class 'dict'>
in loop, col_dict is a:  <class 'dict'> {'colour': 'colour', 'size': [452, 144], 'padding': 5}



Traceback (most recent call last):
  File "/its/home/nn268/antvis/antvis/optics/fns4wandb.py", line 259, in pipeline
    train_model(model, train_loader, val_loader, loss_fn, config, col_dict, device)
  File "/its/home/nn268/antvis/antvis/optics/fns4wandb.py", line 218, in train_model
    t_loss, predict_list, t_num_correct, model, optimizer = loop(model, train_loader, epoch, loss_fn, device, col_dict, config.num_classes, optimizer=optimizer, scheduler=scheduler) #model, x_train, y_train, epoch, loss_fn, device, col_dict, config.num_classes, optimizer=optimizer, scheduler=scheduler
  File "/its/home/nn268/antvis/antvis/optics/loop_fns.py", line 103, in loop
    loss.backward()
  File "/its/home/nn268/.local/lib/python3.10/site-packages/torch/_tensor.py", line 492, in backward
    torch.autograd.backward(
  File "/its/home/nn268/.local/lib/python3.10/site-packages/torch/autograd/__init__.py", line 251, in backward
    Variable._execution_engine.run_backward(  # Calls into the C++ engin

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.48 GiB. GPU 0 has a total capacty of 23.65 GiB of which 160.75 MiB is free. Process 29726 has 7.08 GiB memory in use. Including non-PyTorch memory, this process has 16.40 GiB memory in use. Of the allocated memory 14.22 GiB is allocated by PyTorch, and 1.71 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
"""def tr(config=None):
    with wandb.init(config=config):
        config = wandb.config
        device = "cuda:0" if torch.cuda.is_available() else "cpu"
        model = hp_sweep(config, col_dict, save_dict, device)

wandb.agent(sweep_id, tr, count=2)"""